In [1]:
import sqlite3
# I have used copilot
cursor = sqlite3.connect('exer_4.db')
cursor.execute('CREATE TABLE IF NOT EXISTS Books (BookID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, Title TEXT, Author TEXT, ISBN integer, Status integer)')
cursor.execute('CREATE TABLE IF NOT EXISTS Users (UserID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, Name TEXT, Email TEXT)')
cursor.execute('CREATE TABLE IF NOT EXISTS Reservations (ReservationID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, BookID INTEGER, UserID INTEGER, ReservationDate TEXT)')


In [27]:
print('''Welcome to the library!
      Please choose one of the following options:
      1. Add a new book
      2. Find a book
      3. Check reservation status
      4. Find all books
      5. Modify book details
      6. Delete a book
      7. exit
      ''')

choice = int(input('Enter your choice: '))
if choice == 1:
    cursor.execute('INSERT INTO Books (Title, Author, ISBN, Status) VALUES (?, ?, ?, ?)', (input('Enter the Title: '), input('Enter the Author: '), input('Enter the ISBN: '), 1))
    cursor.commit()

elif choice == 2:
    result = cursor.execute('select Status, Users.UserID, Name, Email from Books left join Reservations on Books.BookID = Reservations.BookID left join Users on Reservations.UserID = Users.UserID where Books.BookID = ?', (input('Enter the BookID: '),)).fetchall()
    if result[0][0] == 1:
      print('The book is available for reservation')
    else:
      print('The book is not available for reservation')
      print('The book is reserved by', result[0][2], 'with email', result[0][3])

elif choice == 3:
   parameter = input('Enter the parameter: ')
   if parameter.startswith('LB'):
      parameter = parameter[2:]
      result = cursor.execute('select Status from Books left join Reservations on Books.BookID = Reservations.BookID left join Users on Reservations.UserID = Users.UserID where Reservations.BookID = ?', (parameter,)).fetchall()
   
      print(result[0][0])
   elif parameter.startswith('LU'):
      parameter = parameter[2:]
      result = cursor.execute('select Status from Books left join Reservations on Books.BookID = Reservations.BookID left join Users on Reservations.UserID = Users.UserID where Reservations.UserID = ?', (parameter,)).fetchall()
    
      print(result[0][0])      
   elif parameter.startswith('LR'):
      parameter = parameter[2:]
      result = cursor.execute('select Status from Books left join Reservations on Books.BookID = Reservations.BookID left join Users on Reservations.UserID = Users.UserID where ReservationID = ?', (parameter,)).fetchall()
      print(result[0][0])
   else:
      result = cursor.execute('select Status from Books left join Reservations on Books.BookID = Reservations.BookID left join Users on Reservations.UserID = Users.UserID where Title = ?', (parameter,)).fetchall()
      if result[0][0] == 1:
         print(result[0][0])
      else:
         print('Not found')
         
elif choice == 4:
   result = cursor.execute('select * from Books').fetchall()
   print(result)

elif choice == 5:
   Book_ID = int(input('Enter the BookID: '))
   Alter = input('Enter the parameter you want to alter and use , to split them: ')
   Alter = Alter.split(',')
   print(Alter)
   if Alter.__contains__('Status') == False:
      for i in Alter:
         cursor.execute(f'UPDATE Books SET {i} = ? WHERE BookID = ?', (input('Enter the new value: '), Book_ID))
   if Alter.__contains__('Status') == True:
      b_or_r = int(input('Enter 0 for borrow and 1 for return: '))
      if b_or_r == 1:
         cursor.execute('UPDATE Books SET Status = ? WHERE BookID = ?', (b_or_r, Book_ID))
         cursor.execute('DELETE FROM Reservations WHERE BookID = ?', (Book_ID,))
      elif b_or_r == 0:
         cursor.execute('UPDATE Books SET Status = ? WHERE BookID = ?', (b_or_r, Book_ID))
         name = input('Enter the name: ')
         email = input('Enter the email: ')
         cursor.execute('INSERT into Users (Name, Email) VALUES (?, ?)', (name,email))
         user_id = cursor.execute('SELECT UserID FROM Users WHERE Name = ? AND Email = ?', (name, email))
         result = cursor.execute('SELECT UserID FROM Users WHERE Name = ? AND Email = ?', (name, email)).fetchone()  
         user_id = result[0]
         cursor.execute('INSERT into Reservations (BookID, UserID, ReservationDate) VALUES (?, ?, ?)', (Book_ID, user_id, input('Enter the ReservationDate: ')))

elif choice == 6:
      cursor.execute('DELETE FROM Books WHERE BookID = ?', (input('Enter the BookID: '),))

elif choice == 7:
      exit()

cursor.commit()

Welcome to the library!
      Please choose one of the following options:
      1. Add a new book
      2. Find a book
      3. Check reservation status
      4. Find all books
      5. Modify book details
      6. Delete a book
      7. exit
      


['Status', 'Title']


In [28]:
#  Use the Alice’s Adventures in Wonderland text from the nltk library’s gutenberg module to conduct topic modeling
from nltk.corpus import gutenberg
alice = gutenberg.sents('carroll-alice.txt')
# do topic modelling
from gensim import corpora
dictionary = corpora.Dictionary(alice)
corpus = [dictionary.doc2bow(text) for text in alice]
import gensim
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word = dictionary, passes = 20)
print(ldamodel.print_topics(num_topics = 3, num_words = 3))
# visulize the topics using pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
vis

# This part I use command and copilot to finish the code

[(0, '0.038*"," + 0.027*"the" + 0.022*"\'"'), (1, '0.110*"\'" + 0.040*"," + 0.037*"I"'), (2, '0.068*"," + 0.055*"the" + 0.035*"and"')]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.137161  0.041834       1        1  52.630824
1      0.031188 -0.136007       2        1  39.601593
0      0.105974  0.094173       3        1   7.767583, topic_info=    Term         Freq        Total Category  logprob  loglift
0      '  1583.000000  1583.000000  Default  30.0000  30.0000
14     I   499.000000   499.000000  Default  29.0000  29.0000
20     ,  1846.000000  1846.000000  Default  28.0000  28.0000
21    ,'   365.000000   365.000000  Default  27.0000  27.0000
42    of   460.000000   460.000000  Default  26.0000  26.0000
..   ...          ...          ...      ...      ...      ...
132  all    11.657545   160.487621   Topic3  -5.4263  -0.0671
76   for    10.966892   129.861783   Topic3  -5.4873   0.0836
39    it    12.662331   490.269504   Topic3  -5.3436  -1.1011
15     -    10.542739   131.208885   Topic3  -5.5268   0.0339
116   !'    10.232911   254.585275   Topic3  -5.5566  -0.6588

[223 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
98        1  0.157221     !
98        2  0.678909     !
98        3  0.164367     !
116       1  0.003928    !'
116       2  0.954494    !'
...     ...       ...   ...
306       2  0.901577   you
306       3  0.076459   you
571       1  0.021254  your
571       2  0.658869  your
571       3  0.318807  your

[308 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])